# Pre processing UCI-HAR

This notebook will pre processing the dataset UCI-HAR (https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)

1. Load the uci dataset
2. Remove the overlap from samples
3. Change the accelerometer mensure from g to m/s²

In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

## Filtering the accelerometer signal

In [2]:
from scipy import signal

def filtering(sig):
    
    sos = signal.butter(3, 0.3, 'low', fs=50, output='sos')
    filtered = signal.sosfilt(sos, sig)
    return filtered

def resample(sig, time_window):
    return signal.resample(sig, 20*time_window)

In [3]:
def transformar(X, y, user, sensor, filter_the_signal=True):
    '''
    Essa função é responsável por pegar os dados do UCI-HAR
    (recortados em janelas de 2,56 segundos e com sobreposição de 50%)

    '''
    
    # Vamos ver quando ocorre uma nova amostra (ou seja, quando não ocorre sobreposição)
    aux = np.zeros(len(y)-1)
    for i in range(len(y)-1):
        aux[i] = np.all(X[i,64:]!=X[i+1,:64])
    ind = np.nonzero(aux)[0]+1
    ind = np.append([0],ind)
    ind = np.append(ind,[len(y)])
    
    # Agora vamos construir as capturas originais (sem sobreposição)
    sigs = []
    yy = []
    uuser = []
    for i in range(len(ind)-1):
        sig = X[ind[i]:ind[i+1],64:].reshape((ind[i+1]-ind[i])*64)
        sig = np.append(X[ind[i],:64],sig)
        
        if sensor < 3:
            if filter_the_signal:
                sig_filtered = sig - filtering(sig) # Removing the gravity
            else:
                sig_filtered = sig
            time_window = len(sig_filtered) // 50 # The time window is the time of the total window
#             sig_resampled = resample(sig_filtered, time_window) # Resampling the signal
            sig_resampled = sig
    
            sigs.append(sig_resampled)
            yy.append(y[ind[i]])
            uuser.append(user[ind[i]])
            
        else:
            time_window = len(sig) // 50 # The time window is the time of the total window
#             sig_resampled = resample(sig, time_window) # Resampling the signal

            sigs.append(sig)
            yy.append(y[ind[i]])
            uuser.append(user[ind[i]])
#         print('Time window: ', time_window)
    # Vamos construir as janelas de 3 segundos (lembrando que o UCI-HAR está amostrado a uma taxa de 20 Hz)
#     new_time_window = 3
#     sample_rate = 20
#     points_per_new_window = new_time_window*sample_rate

    print('Time window: ', time_window, len(sig) / 50)
    points_per_window = 50 * time_window
    print(points_per_window)
    
    aux = np.zeros(len(sigs))
    for i in range(len(sigs)):
        aux[i] = len(sigs[i])
    Na = np.sum(aux//points_per_window, dtype=np.int64)
    mat = np.zeros((Na,points_per_window))
    yyy = np.zeros(Na)
    uuuser = np.zeros(Na)
    k = 0
    for i in range(len(sigs)):
        for j in range(len(sigs[i])//points_per_window):
            mat[k,:] = sigs[i][j*points_per_window:(j+1)*points_per_window]
            yyy[k] = yy[i]
            uuuser[k] = uuser[i]
            k+=1
    return mat, yyy, uuuser;

## Pre processing the train data

In [4]:
# Vamos abrir os dados de treino do UCI-HAR
pasta = "../../../../../Downloads/UCI HAR Dataset/UCI HAR Dataset/train/"
# dados = [None]*6
sensor = [
    'Inertial Signals/total_acc_x_train.txt', 
    'Inertial Signals/total_acc_y_train.txt', 
    'Inertial Signals/total_acc_z_train.txt', 
    'Inertial Signals/body_gyro_x_train.txt', 
    'Inertial Signals/body_gyro_y_train.txt', 
    'Inertial Signals/body_gyro_z_train.txt'
]
dados = [np.loadtxt(pasta+sensor[i]) for i in range(6)]
                    
# dados[0] = np.loadtxt(pasta+'Inertial Signals/total_acc_x_train.txt')
# dados[1] = np.loadtxt(pasta+'Inertial Signals/total_acc_y_train.txt')
# dados[2] = np.loadtxt(pasta+'Inertial Signals/total_acc_z_train.txt')
# dados[3] = np.loadtxt(pasta+'Inertial Signals/body_gyro_x_train.txt')
# dados[4] = np.loadtxt(pasta+'Inertial Signals/body_gyro_y_train.txt')
# dados[5] = np.loadtxt(pasta+'Inertial Signals/body_gyro_z_train.txt')
                    
y = np.loadtxt(pasta+'y_train.txt')
user = np.loadtxt(pasta+'subject_train.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 1226
WALKING_UPSTAIRS: 1073
WALKING_DOWNSTAIRS: 986
SITTING: 1286
STANDING: 1374
LAYING: 1407


In [5]:
new_time_window = 3
sample_rate = 50
points_per_new_window = new_time_window*sample_rate

train = []
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user, i, filter_the_signal=False)
    if type(train) == list:
        train = mat
    else:
        train = np.concatenate([train, mat], axis=1)
    print(train.shape)
train = np.column_stack((train,yyy.astype(int),uuuser.astype(int)))

Time window:  14 14.08
700
(395, 700)
Time window:  14 14.08
700
(395, 1400)
Time window:  14 14.08
700
(395, 2100)
Time window:  14 14.08
700
(395, 2800)
Time window:  14 14.08
700
(395, 3500)
Time window:  14 14.08
700
(395, 4200)


In [6]:
time_window = 14
sample_rate = 50
points_per_new_window = time_window*sample_rate

colunas = ['accel-x-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['activity code']
colunas += ['user']

In [7]:
df_train = pd.DataFrame(data=train)

In [8]:
df_train.columns = colunas
df_train = df_train.astype({"activity code": int, "user": int})

# Removing the activity laying
df_train = df_train[df_train['activity code'].isin([1, 2, 3, 4, 5])]
df_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-692,gyro-z-693,gyro-z-694,gyro-z-695,gyro-z-696,gyro-z-697,gyro-z-698,gyro-z-699,activity code,user
0,1.012817,1.022833,1.022028,1.017877,1.023680,1.016974,1.017746,1.019263,1.016417,1.020745,...,-0.004949,-0.003745,-0.006392,-0.003554,0.002851,-0.000797,-0.000314,0.002297,5,1
1,1.005814,1.006548,1.003347,1.002136,0.997458,1.001599,1.006555,1.002259,1.000160,1.000095,...,-0.009056,-0.010042,-0.004734,-0.005272,-0.003232,-0.001824,-0.000469,0.003222,5,1
2,0.906832,0.909364,0.908368,0.911149,0.924365,0.925115,0.919907,0.911792,0.902463,0.916602,...,-0.003253,-0.000051,0.002566,0.001496,-0.001453,-0.004019,-0.004448,-0.001147,4,1
3,0.952890,0.958077,0.949529,0.945041,0.952956,0.955458,0.955231,0.956717,0.960898,0.957649,...,-0.005677,-0.004537,-0.005054,-0.004893,-0.003993,-0.003279,-0.004147,-0.008459,4,1
6,1.046428,1.051911,1.079723,1.152605,1.243663,1.347546,1.394581,1.248929,0.973286,0.781757,...,-0.082830,-0.338226,-0.260156,-0.075749,-0.130901,-0.155239,-0.353205,-0.598767,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1.145192,1.109868,1.061991,1.012688,0.926216,0.871875,0.827314,0.789831,0.796183,0.806194,...,0.378695,0.547155,0.613313,0.630506,0.696747,0.737468,0.761132,0.835463,1,30
391,0.838206,0.834605,0.816020,0.774153,0.718732,0.697873,0.729302,0.766743,0.800491,0.794401,...,-0.170144,-0.070931,0.153594,0.439557,0.488488,0.502762,0.523901,0.217399,3,30
392,1.364822,1.340470,1.296644,1.252721,1.179719,1.073125,1.038065,1.077064,1.087642,1.058006,...,0.086359,0.152076,0.187116,0.275017,0.401570,0.442173,0.307598,0.166505,2,30
393,1.034126,1.017101,1.010441,1.022527,1.054362,1.079674,1.121693,1.190717,1.269983,1.406678,...,0.405498,0.468114,0.520275,0.567072,0.568914,0.571440,0.540665,0.471231,2,30


In [9]:
print('Número de amostras por classe na formatação final:')
yyy = np.array(df_train['activity code'])
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 85
WALKING_UPSTAIRS: 17
WALKING_DOWNSTAIRS: 8
SITTING: 90
STANDING: 94


In [10]:
df_train.iloc[:,:2101]

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,accel-z-691,accel-z-692,accel-z-693,accel-z-694,accel-z-695,accel-z-696,accel-z-697,accel-z-698,accel-z-699,gyro-x-0
0,1.012817,1.022833,1.022028,1.017877,1.023680,1.016974,1.017746,1.019263,1.016417,1.020745,...,0.068823,0.077640,0.079022,0.078201,0.083354,0.087636,0.086667,0.083782,0.083862,0.030191
1,1.005814,1.006548,1.003347,1.002136,0.997458,1.001599,1.006555,1.002259,1.000160,1.000095,...,-0.018128,-0.016993,-0.016850,-0.015000,-0.012564,-0.012339,-0.011707,-0.011451,-0.012373,-0.060814
2,0.906832,0.909364,0.908368,0.911149,0.924365,0.925115,0.919907,0.911792,0.902463,0.916602,...,0.459491,0.457365,0.454216,0.450142,0.449919,0.456117,0.458956,0.454240,0.448947,0.058535
3,0.952890,0.958077,0.949529,0.945041,0.952956,0.955458,0.955231,0.956717,0.960898,0.957649,...,0.311113,0.318208,0.318797,0.316502,0.315333,0.313479,0.309420,0.304678,0.302941,-0.217883
6,1.046428,1.051911,1.079723,1.152605,1.243663,1.347546,1.394581,1.248929,0.973286,0.781757,...,-0.072654,-0.008315,0.030196,0.016445,-0.043099,-0.113193,-0.194504,-0.205016,-0.202202,-0.694557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1.145192,1.109868,1.061991,1.012688,0.926216,0.871875,0.827314,0.789831,0.796183,0.806194,...,-0.090602,-0.112646,-0.093540,-0.100344,-0.086478,-0.039207,-0.017440,0.018032,0.016679,-0.958940
391,0.838206,0.834605,0.816020,0.774153,0.718732,0.697873,0.729302,0.766743,0.800491,0.794401,...,0.195379,0.172390,0.115758,0.034585,-0.043605,-0.096286,-0.169965,-0.296930,-0.357755,-0.763525
392,1.364822,1.340470,1.296644,1.252721,1.179719,1.073125,1.038065,1.077064,1.087642,1.058006,...,-0.112628,-0.101219,-0.099735,-0.096485,-0.102029,-0.142855,-0.200928,-0.255455,-0.273849,0.586928
393,1.034126,1.017101,1.010441,1.022527,1.054362,1.079674,1.121693,1.190717,1.269983,1.406678,...,-0.382645,-0.412329,-0.392004,-0.348973,-0.258787,-0.178031,-0.183428,-0.197017,-0.191813,-0.862893


In [11]:
# Mudando a unidade de medida do acelerômetro para m/s² (Basta multiplicar por g = 9.81)
# g = 9.81
g = 9.81
df_train.iloc[:,:2100] = df_train.iloc[:,:2100]*g
df_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-692,gyro-z-693,gyro-z-694,gyro-z-695,gyro-z-696,gyro-z-697,gyro-z-698,gyro-z-699,activity code,user
0,9.935735,10.033992,10.026095,9.985373,10.042301,9.976515,9.984088,9.998970,9.971051,10.013508,...,-0.004949,-0.003745,-0.006392,-0.003554,0.002851,-0.000797,-0.000314,0.002297,5,1
1,9.867035,9.874236,9.842834,9.830954,9.785064,9.825686,9.874305,9.832161,9.811570,9.810932,...,-0.009056,-0.010042,-0.004734,-0.005272,-0.003232,-0.001824,-0.000469,0.003222,5,1
2,8.896019,8.920862,8.911093,8.938377,9.068020,9.075381,9.024289,8.944681,8.853163,8.991866,...,-0.003253,-0.000051,0.002566,0.001496,-0.001453,-0.004019,-0.004448,-0.001147,4,1
3,9.347849,9.398732,9.314881,9.270848,9.348495,9.373047,9.370815,9.385395,9.426407,9.394534,...,-0.005677,-0.004537,-0.005054,-0.004893,-0.003993,-0.003279,-0.004147,-0.008459,4,1
6,10.265459,10.319247,10.592083,11.307055,12.200334,13.219426,13.680840,12.251993,9.547935,7.669040,...,-0.082830,-0.338226,-0.260156,-0.075749,-0.130901,-0.155239,-0.353205,-0.598767,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,11.234334,10.887805,10.418132,9.934469,9.086178,8.553098,8.115947,7.748245,7.810558,7.908766,...,0.378695,0.547155,0.613313,0.630506,0.696747,0.737468,0.761132,0.835463,1,30
391,8.222805,8.187475,8.005151,7.594442,7.050757,6.846135,7.154449,7.521751,7.852813,7.793075,...,-0.170144,-0.070931,0.153594,0.439557,0.488488,0.502762,0.523901,0.217399,3,30
392,13.388904,13.150011,12.720078,12.289193,11.573043,10.527356,10.183418,10.565998,10.669768,10.379039,...,0.086359,0.152076,0.187116,0.275017,0.401570,0.442173,0.307598,0.166505,2,30
393,10.144776,9.977761,9.912426,10.030990,10.343291,10.591602,11.003808,11.680934,12.458533,13.799511,...,0.405498,0.468114,0.520275,0.567072,0.568914,0.571440,0.540665,0.471231,2,30


## Pre processing the test data

In [12]:
# Agora vamos abrir os dados de treino do UCI-HAR
pasta = "../../../../../Downloads/UCI HAR Dataset/UCI HAR Dataset/test/"
dados = [None]*6
sensor = [
    'Inertial Signals/total_acc_x_test.txt', 
    'Inertial Signals/total_acc_y_test.txt', 
    'Inertial Signals/total_acc_z_test.txt', 
    'Inertial Signals/body_gyro_x_test.txt', 
    'Inertial Signals/body_gyro_y_test.txt', 
    'Inertial Signals/body_gyro_z_test.txt'
]
dados = [np.loadtxt(pasta+sensor[i]) for i in range(6)]
                    
y = np.loadtxt(pasta+'y_test.txt')
user = np.loadtxt(pasta+'subject_test.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING'
          , 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 496
WALKING_UPSTAIRS: 471
WALKING_DOWNSTAIRS: 420
SITTING: 491
STANDING: 532
LAYING: 537


In [13]:
new_time_window = 14
sample_rate = 50
points_per_new_window = new_time_window*sample_rate

test = []
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user, i, filter_the_signal=False)
    if type(test) == list:
        test = mat
    else:
        test = np.concatenate([test, mat], axis=1)
    print(test.shape)
test = np.column_stack((test,yyy.astype(int),uuuser.astype(int)))

Time window:  14 14.08
700
(165, 700)
Time window:  14 14.08
700
(165, 1400)
Time window:  14 14.08
700
(165, 2100)
Time window:  14 14.08
700
(165, 2800)
Time window:  14 14.08
700
(165, 3500)
Time window:  14 14.08
700
(165, 4200)


In [14]:
df_test = pd.DataFrame(data=test)

In [15]:
df_test.columns = colunas
df_test = df_test.astype({"activity code": int, "user": int})

# Removing the activity laying
df_test = df_test[df_test['activity code'].isin([1, 2, 3, 4, 5])]
df_test

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-692,gyro-z-693,gyro-z-694,gyro-z-695,gyro-z-696,gyro-z-697,gyro-z-698,gyro-z-699,activity code,user
0,1.041216,1.041803,1.039086,1.054768,1.028376,1.006216,0.962391,0.929196,0.943047,0.951635,...,0.005502,0.006019,0.003373,0.002493,0.006035,0.004671,0.003450,0.001553,5,2
1,0.957625,0.965912,0.966474,0.961965,0.955425,0.954355,0.959792,0.962922,0.974986,0.987045,...,0.007234,0.002748,0.000480,0.004775,0.006821,0.006203,0.006346,0.010691,5,2
2,1.002248,1.014939,1.011894,1.008806,1.000582,0.997897,1.009186,1.010335,1.010461,1.011460,...,-0.002492,-0.001172,-0.001218,-0.001821,0.001201,0.000388,-0.000789,0.001041,4,2
3,1.010407,1.013024,1.015286,1.020490,1.021184,1.035372,1.053125,1.049356,1.029576,1.012953,...,-0.002825,-0.000227,-0.001138,-0.005637,-0.005919,-0.004045,0.001676,0.002371,4,2
6,0.945546,0.991955,1.019516,1.065764,1.071036,1.000766,0.956370,0.871978,0.770315,0.785545,...,-0.052351,-0.094570,-0.315063,-0.590469,-0.626321,-0.429448,-0.287951,-0.306121,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0.945157,0.969463,1.059068,1.131712,1.126833,1.073291,1.036205,1.043919,1.041524,1.048385,...,-0.001482,-0.033816,0.182405,0.379443,0.152466,-0.176845,-0.169554,-0.037211,1,24
161,0.863691,0.834085,0.784460,0.739472,0.724078,0.731690,0.762140,0.827458,0.896481,0.952862,...,0.108744,-0.061848,-0.258192,-0.368345,-0.324138,-0.177479,-0.028066,0.066757,3,24
162,1.180778,1.066991,1.025297,0.941798,0.900581,0.915682,0.929387,0.954279,0.920682,0.873033,...,0.251716,0.246032,0.137421,0.069774,0.173882,0.400048,0.606038,0.705266,2,24
163,0.917196,0.933939,0.971307,1.015320,1.036636,0.998075,0.951523,1.010587,1.166020,1.379448,...,0.236642,0.127274,0.049975,0.060117,0.077440,0.066152,0.087770,0.116076,2,24


In [16]:
print('Número de amostras por classe na formatação final:')
yyy = np.array(df_test['activity code'])
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 36
WALKING_UPSTAIRS: 14
WALKING_DOWNSTAIRS: 5
SITTING: 35
STANDING: 39


In [17]:
# Mudando a unidade de medida do acelerômetro para m/s² (Basta multiplicar por g = 9.81)
# g = 9.81
g = 9.81
df_test.iloc[:,:2100] = df_test.iloc[:,:2100]*g
df_test

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-692,gyro-z-693,gyro-z-694,gyro-z-695,gyro-z-696,gyro-z-697,gyro-z-698,gyro-z-699,activity code,user
0,10.214329,10.220087,10.193434,10.347274,10.088369,9.870979,9.441051,9.115415,9.251288,9.335537,...,0.005502,0.006019,0.003373,0.002493,0.006035,0.004671,0.003450,0.001553,5,2
1,9.394298,9.475596,9.481109,9.436872,9.372719,9.362224,9.415563,9.446269,9.564609,9.682913,...,0.007234,0.002748,0.000480,0.004775,0.006821,0.006203,0.006346,0.010691,5,2
2,9.832053,9.956552,9.926680,9.896387,9.815709,9.789373,9.900115,9.911386,9.912622,9.922423,...,-0.002492,-0.001172,-0.001218,-0.001821,0.001201,0.000388,-0.000789,0.001041,4,2
3,9.912093,9.937765,9.959956,10.011007,10.017815,10.156999,10.331156,10.294182,10.100141,9.937069,...,-0.002825,-0.000227,-0.001138,-0.005637,-0.005919,-0.004045,0.001676,0.002371,4,2
6,9.275801,9.731083,10.001452,10.455145,10.506863,9.817514,9.381992,8.554100,7.556790,7.706200,...,-0.052351,-0.094570,-0.315063,-0.590469,-0.626321,-0.429448,-0.287951,-0.306121,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,9.271995,9.510433,10.389457,11.102095,11.054232,10.528985,10.165171,10.240845,10.217350,10.284657,...,-0.001482,-0.033816,0.182405,0.379443,0.152466,-0.176845,-0.169554,-0.037211,1,24
161,8.472813,8.182377,7.695552,7.254219,7.103200,7.177875,7.476589,8.117359,8.794475,9.347580,...,0.108744,-0.061848,-0.258192,-0.368345,-0.324138,-0.177479,-0.028066,0.066757,3,24
162,11.583432,10.467182,10.058164,9.239039,8.834703,8.982836,9.117285,9.361476,9.031890,8.564455,...,0.251716,0.246032,0.137421,0.069774,0.173882,0.400048,0.606038,0.705266,2,24
163,8.997698,9.161940,9.528526,9.960289,10.169399,9.791117,9.334437,9.913858,11.438656,13.532385,...,0.236642,0.127274,0.049975,0.060117,0.077440,0.066152,0.087770,0.116076,2,24


In [18]:
df_train.to_csv('../../data_2/views/UCI-HAR/raw_unbalanced_view_with_gravity_acc_9.81_train_test-v1/train.csv',index=False)
df_test.to_csv('../../data_2/views/UCI-HAR/raw_unbalanced_view_with_gravity_acc_9.81_train_test-v1/test.csv',index=False)